# EMNIST

In [33]:
%load_ext autoreload
%autoreload 2
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset
import wandb
from pytorch_lightning.loggers import WandbLogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)

cuda


In [3]:
# Create the working directory from timestamp and model name
model_name = 'cvae_emnist' # invertible network
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
# workingDir = f'tb_logs/{model_name}_{timestamp}'
# os.mkdir(workingDir)
root='/home/mila/n/nizar.islah/expectation-clamp/'

In [4]:
lr = 1e-3
batch_size = 64
latent_dim = 20
TRAIN_CVAE = True

In [32]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,)),
    lambda x: x.rot90(1,[2,1]).flip(2)
])
dataset = datasets.EMNIST(root=root, download=False, split='byclass', train=True, transform=transform)
new_dataset = partition_dataset(dataset, range(10, 36))
train_set, val_set = torch.utils.data.random_split(new_dataset, [round(0.8*len(new_dataset)), round(0.2*len(new_dataset))])
test_set = datasets.EMNIST(root=root, download=False, split='byclass', train=False, transform=transform)
# Dataloaders

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [6]:
x,t=next(iter(train_loader))
torchvision.utils.save_image(x, 'image.pdf', nrow=8)
print(['abcdefghijklmnopqrstuvwxyz'[i] for i in t])

['h', 'f', 'm', 'x', 'm', 'c', 'c', 'e', 'a', 't', 'r', 'p', 'k', 'c', 'o', 'a', 'c', 'i', 'a', 'n', 'r', 'n', 'o', 'w', 'i', 't', 'o', 'c', 's', 'd', 'o', 'c', 'b', 'p', 'z', 'o', 'f', 'i', 'i', 'p', 'u', 'e', 'h', 's', 'x', 'm', 'z', 'a', 'o', 'd', 'm', 'w', 'h', 'u', 'o', 'd', 's', 't', 'h', 'w', 'y', 'r', 's', 's']


In [34]:
#tb_logger = TensorBoardLogger('tb_logs', name=model_name)

enc_layers = [28*28, 1024, 512]
dec_layers = [512, 1024, 28*28]
if TRAIN_CVAE:
    wandb.init(project="EMNIST_CVAE", entity="team-nizar")
    wandb_logger = WandbLogger(project="EMNIST_CVAE", log_model="all")
    model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
    trainer = pl.Trainer(gpus=1,logger=wandb_logger,max_epochs=100,auto_lr_find=True)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)

Loss/Train,█▃▃▂▂▂▂▁▁▁▁▁▁▁
Loss/TrainStream,█▆▄▄▄▄▃▃▂▃▂▃▂▃▂▂▂▂▂▂▂▂▂▃▂▃▃▂▂▃▁▂▁▂▂▁▁▂▁▂
Loss/Val,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/ValStream,▆▇█▅▆█▇▅▅▆▄▅▃▅▄▄▃▄▃▄▂▃▄▃▃▃▄▃▃▁▃▃▂▂▃▃▃▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss/Train,123.42824
Loss/TrainStream,128.43784
Loss/Val,125.87069
Loss/ValStream,138.92708
trainer/global_step,34519


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | EMNIST_Encoder | 1.4 M 
1 | decoder | EMNIST_Decoder | 1.4 M 
-------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.915    Total estimated model params size (MB)
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/mila/n/nizar.islah/ambiguous-dataset/amb

Validation sanity check: 0it [00:00, ?it/s]

/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: Futu

...

/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'mu', 'targets'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


Validating: 0it [00:00, ?it/s]

/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'mu', 'targets'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


Validating: 0it [00:00, ?it/s]

/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/mila/n/nizar.islah/test/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [40]:
model.eval()

def gen_imgs(model, n_samples, n_latent, n_classes=26):
    targets = torch.randint(0, n_classes, (n_samples,))
    c = torch.zeros(n_samples, n_classes).to(device)
    c[range(n_samples), targets] = 1
    z = torch.randn(n_samples, n_latent).to(device)
    rec = model.decoder(z, c).view(-1, 1, 28, 28)
    torchvision.utils.save_image(rec, "reconstruction.pdf", nrow=8)
    return

gen_imgs(model, 64, latent_dim)

In [42]:
model

EMNIST_CVAE(
  (encoder): EMNIST_Encoder(
    (MLP): Sequential(
      (L0): Linear(in_features=810, out_features=1024, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=1024, out_features=512, bias=True)
      (A1): ReLU()
    )
    (fc_mu): Linear(in_features=512, out_features=20, bias=True)
    (fc_logvar): Linear(in_features=512, out_features=20, bias=True)
  )
  (decoder): EMNIST_Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=46, out_features=512, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=512, out_features=1024, bias=True)
      (A1): ReLU()
      (L2): Linear(in_features=1024, out_features=784, bias=True)
      (sigmoid): Sigmoid()
    )
  )
)

In [41]:
torch.save(model,"emnist_cvae.pth")

PicklingError: Can't pickle <function <lambda> at 0x7f78400cf940>: attribute lookup <lambda> on __main__ failed